# Skin lesion classification of dermoscopic images using machine learning and convolutional neural network

19 December 2022

https://www.nature.com/articles/s41598-022-22644-9#Tab7

https://aihub.or.kr/aihubdata/data/view.do?currMenu=&topMenu=&aihubDataSe=realm&dataSetSn=561

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import cv2
import glob
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import tensorflow as tf
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from tensorflow.keras import models
from tensorflow.keras.layers import BatchNormalization, Conv2D, Dense, Dropout, Flatten, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tqdm import tqdm

In [3]:
def get_image_and_json_paths(src_path):
    image_paths = sorted(glob.glob(os.path.join(src_path, '**', '*.jpg'), recursive=True))
    json_paths = sorted(glob.glob(os.path.join(src_path, '**', '*.json'), recursive=True))

    return image_paths, json_paths

def extract_data_from_json(json_paths):
    lesions, polygon_locations, box_locations = [], [], []
    for json_path in tqdm(json_paths, desc='Loading JSON', unit=' file'):
        try:
            with open(json_path, 'r', encoding='utf-8') as file:
                json_data = json.loads(re.sub(r'[\x00-\x1F\x7F-\x9F]', '', file.read()))
                metadata, labeling_info = json_data.get('metaData', None), json_data['labelingInfo']

                lesions.append(metadata.get('lesions', None))

                for entry in labeling_info:
                    if 'polygon' in entry:
                        polygon_locations.extend(entry['polygon'].get('location', None))
                    if 'box' in entry:
                        box_locations.extend(entry['box'].get('location', None))

        except Exception as e:
            print(json_path)
            print(e)

    return lesions, polygon_locations, box_locations

def combine_images_and_masks(image_list, mask_list):
    combined_images = []

    for i in tqdm(range(len(image_list)), desc='Combining Images and Masks'):
        image = image_list[i]
        mask = mask_list[i]

        mask_channel = np.zeros_like(image[:, :, 0], dtype=np.uint8)

        mask_channel[mask == 1] = 255

        combined_image = np.dstack((image, mask_channel))

        combined_images.append(combined_image)

    return combined_images

def one_hot_encode_labels(labels):
    label_binarizer = LabelBinarizer()
    encoded_labels = label_binarizer.fit_transform(labels)
    return np.array(encoded_labels), label_binarizer

def create_polygon_binary_masks(image_list, polygon_locations_list):
    binary_masks = []

    for image, polygon_locations in tqdm(zip(image_list, polygon_locations_list), desc='Generating Binary Masks', total=len(image_list)):
        mask = np.zeros(image.shape[:2], dtype=np.uint8)

        for poly_coords in polygon_locations:
            if poly_coords:
                poly_points = []
                i = 1
                while f'x{i}' in poly_coords and f'y{i}' in poly_coords:
                    x = poly_coords[f'x{i}']
                    y = poly_coords[f'y{i}']
                    poly_points.append([x, y])
                    i += 1
                if len(poly_points) > 0:
                    poly_points = np.array(poly_points, dtype=np.int32)
                    cv2.fillPoly(mask, [poly_points], 255)
            print(poly_coords)

        binary_masks.append(mask)

    return binary_masks

def visualize_masks(image_data, mask_data, selected_indices, figsize=(10, 5)):
    for idx in selected_indices:
        image = image_data[idx]
        mask_map = mask_data[idx]

        plt.figure(figsize=figsize)
        plt.subplot(1, mask_map.shape[2] + 1, 1)
        plt.imshow(image)
        plt.title('Original Image')

        for ch in range(mask_map.shape[2]):
            plt.subplot(1, mask_map.shape[2] + 1, ch + 2)
            plt.imshow(mask_map[:, :, ch], cmap='gray')
            plt.title(f'Channel {ch}')

        plt.tight_layout()
        plt.show()

# camera cat

In [4]:
src_path = '/content/drive/Shareddrives/반려묘/일반카메라'

In [5]:
image_paths, json_paths = get_image_and_json_paths(src_path)

In [6]:
lesions, polygon_locations, box_locations = extract_data_from_json(json_paths)

Loading JSON: 100%|██████████| 15000/15000 [00:11<00:00, 1272.33 file/s]


In [7]:
lesions[:5]

['A7', 'A7', 'A7', 'A7', 'A7']

In [8]:
polygon_locations[0]

{'x1': 370,
 'y1': 291,
 'x2': 311,
 'y2': 322,
 'x3': 310,
 'y3': 338,
 'x4': 415,
 'y4': 336,
 'x5': 515,
 'y5': 281,
 'x6': 607,
 'y6': 246,
 'x7': 647,
 'y7': 222,
 'x8': 628,
 'y8': 221,
 'x9': 548,
 'y9': 221,
 'x10': 370,
 'y10': 291}

In [9]:
box_locations[:5]

[{'x': 310, 'y': 221, 'width': 337, 'height': 117},
 {'x': 1295, 'y': 605, 'width': 310, 'height': 201},
 {'x': 944, 'y': 384, 'width': 174, 'height': 72},
 {'x': 620, 'y': 412, 'width': 56, 'height': 112},
 {'x': 1261, 'y': 298, 'width': 75, 'height': 123}]

In [10]:
df = pd.DataFrame({'image_path': image_paths, 'lesion': lesions})

df

,image_path,lesion
0,/content/drive/Shareddrives/반려묘/일반카메라/무증상...,A7
1,/content/drive/Shareddrives/반려묘/일반카메라/무증상...,A7
2,/content/drive/Shareddrives/반려묘/일반카메라/무증상...,A7
3,/content/drive/Shareddrives/반려묘/일반카메라/무증상...,A7
4,/content/drive/Shareddrives/반려묘/일반카메라/무증상...,A7
...,...,...
14995,/content/drive/Shareddrives/반려묘/일반카메라/유증상...,A6
14996,/content/drive/Shareddrives/반려묘/일반카메라/유증상...,A6
14997,/content/drive/Shareddrives/반려묘/일반카메라/유증상...,A6
14998,/content/drive/Shareddrives/반려묘/일반카메라/유증상...,A6


In [11]:
datagen = ImageDataGenerator(rescale=1.0/255)

In [12]:
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

In [13]:
train_generator = datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='image_path',
    y_col='lesion',
    class_mode='categorical',
    target_size=(96, 96),
)

Found 12000 validated image filenames belonging to 4 classes.


In [14]:
val_generator = datagen.flow_from_dataframe(
    dataframe=val_df,
    x_col='image_path',
    y_col='lesion',
    class_mode='categorical',
    target_size=(96, 96),
)

Found 1500 validated image filenames belonging to 4 classes.


In [15]:
test_generator = datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col='image_path',
    y_col='lesion',
    class_mode='categorical',
    target_size=(96, 96),
    shuffle=False
)

Found 1500 validated image filenames belonging to 4 classes.


In [16]:
model = models.Sequential()

model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same', input_shape=(96,96,3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(3, 3)))

model.add(Dropout(0.25))

model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dropout(0.25))

model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(units=1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(units=4, activation='softmax'))

opt = Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [ ]:
history1 = model.fit(
    train_generator,
    epochs=150,
    batch_size=64,
    validation_data=val_generator
)

Epoch 1/150
 96/375 [======>.......................] - ETA: 21:35 - loss: 1.1007 - accuracy: 0.5723

In [ ]:
y_pred = model.predict(test_generator)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = test_generator.classes

print(classification_report(y_true_classes, y_pred_classes))

In [ ]:
# TensorFlow Lite 모델로 변환
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# 변환된 모델을 파일로 저장
with open('72.tflite', 'wb') as f:
    f.write(tflite_model)

In [ ]:
import cv2
import numpy as np
from skimage.feature import greycomatrix
import h5py

class ImageFeatureExtractor:
    def __init__(self, target_size=(96, 96)):
        self.target_size = target_size

    def preprocess_image(self, image_path):
        image = cv2.imread(image_path)
        image = cv2.resize(image, self.target_size)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        return image

    def extract_color_histogram(self, image):
        histogram = cv2.calcHist([image], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
        histogram = cv2.normalize(histogram, histogram).flatten()
        return histogram

    def extract_hu_moments(self, image):
        gray_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        moments = cv2.HuMoments(cv2.moments(gray_image)).flatten()
        return moments

    def extract_haralick_texture(self, image):
        gray_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        texture = greycomatrix(gray_image, distances=[1], angles=[0], symmetric=True, normed=True)
        haralick_features = np.mean(np.array(greycomatrix_features), axis=(0, 1, 2))
        return haralick_features

    def extract_features(self, image_path):
        image = self.preprocess_image(image_path)
        color_histogram = self.extract_color_histogram(image)
        hu_moments = self.extract_hu_moments(image)
        haralick_texture = self.extract_haralick_texture(image)
        global_features = np.concatenate([color_histogram, hu_moments, haralick_texture])
        return global_features

    def save_features_to_hdf5(self, features, labels, output_path):
        with h5py.File(output_path, 'w') as f:
            f.create_dataset('features', data=features)
            f.create_dataset('labels', data=labels)

# Usage
json_paths = [...]  # List of JSON file paths
image_paths = [...]  # List of image file paths
labels = [...]  # List of corresponding labels

feature_extractor = ImageFeatureExtractor(target_size=(96, 96))

features = []
for image_path in image_paths:
    image_features = feature_extractor.extract_features(image_path)
    features.append(image_features)

features = np.array(features)
labels = np.array(labels)

output_path = 'features.h5'
feature_extractor.save_features_to_hdf5(features, labels, output_path)

ValueError: ignored

In [ ]:
train_images, test_images, train_labels, test_labels = train_test_split(combined_df, encoded_labels, test_size=0.2, random_state=42)
test_images, val_images, test_labels, val_labels = train_test_split(test_images, test_labels, test_size=0.5, random_state=42)

In [ ]:
history2 = model.fit(train_images, train_labels, epochs=150, batch_size=32, validation_data=(val_images, val_labels))

In [ ]:
y_pred = model.predict(test_images)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(test_labels, axis=1)

print(classification_report(y_true_classes, y_pred_classes))

# microscope cat

In [ ]:
from tensorflow.keras.layers import PReLU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.layers import Concatenate, MaxPooling2D, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import concatenate
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.losses import mean_squared_error

def inception_module(input_layer, filters):
    conv1x1 = Conv2D(filters[0], (1, 1), activation='relu')(input_layer)
    conv3x3_reduce = Conv2D(filters[1], (1, 1), activation='relu')(input_layer)
    conv3x3 = Conv2D(filters[2], (3, 3), padding='same', activation='relu')(conv3x3_reduce)
    conv5x5_reduce = Conv2D(filters[3], (1, 1), activation='relu')(input_layer)
    conv5x5 = Conv2D(filters[4], (5, 5), padding='same', activation='relu')(conv5x5_reduce)
    maxpool = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(input_layer)
    maxpool_conv = Conv2D(filters[5], (1, 1), activation='relu')(maxpool)
    inception_output = Concatenate(axis=-1)([conv1x1, conv3x3, conv5x5, maxpool_conv])
    return inception_output

# Input
input_shape = (128, 128, 3)
input_layer = Input(shape=input_shape)

# Inception block
inception_output = inception_module(input_layer, filters=[64, 128, 192, 32, 96, 64])
inception_output = inception_module(inception_output, filters=[64, 128, 192, 32, 96, 64])
# Add more inception modules if needed

# Primary Capsule layer
primary_capsules = Conv2D(32, (1, 1), activation='relu')(inception_output)

# Higher Capsule layers
# (Add imperative routing mechanism layers here)

# PReLU activation for routing
higher_capsules_prelu = PReLU()(higher_capsules)

# Flatten and Fully Connected layers
capsule_flatten = Flatten()(higher_capsules_prelu)  # Flatten higher capsules
output_layer = Dense(2, activation='softmax')(capsule_flatten)  # Two capsules: parasitized and uninfected

# Create the model
model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model with Adam optimizer and custom loss function
optimizer = Adam(learning_rate=0.007, beta_1=0.8)
loss_fn = custom_loss_function # Define the custom loss function as described in the paper
model.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy'])

# Print the model summary
model.summary()